In [1]:
import pyemu
import os, shutil
import pandas as pd
import flopy
import numpy as np
import platform

# This notebook is designed to show high-level PST file generation and manipulation examples, and also highlight some of the steps that `Schur` object functions perform in a bit more detail

The assumption is that a user has a model that `flopy` can grok. That's all we need, using the BOSS PEST-ification techniques `pyemu` has. 

In [2]:
nam_file = "freyberg.nam"
org_model_ws = "Freyberg_transient"
new_model_ws = "pest_setup"
EXE_DIR = 'bin'
if os.path.exists(EXE_DIR):
    if "window" in platform.platform().lower():
        exe_files = [f for f in os.listdir(EXE_DIR) if f.endswith('exe')]
    else:
        exe_files = [f for f in os.listdir(EXE_DIR) if not f.endswith('exe')]
    [shutil.copy2(os.path.join(EXE_DIR,f),os.path.join('temp',f)) for f in exe_files]
# load the model, change dir and run once to get a hydmod output file and list file
m = flopy.modflow.Modflow.load(nam_file,model_ws=org_model_ws,check=False)
m.change_model_ws("temp",reset_external=True)
m.name = nam_file.split(".")[0]

# let's just retain the calibration data for now by trimming HYDMOD
hyd = m.get_package('hyd')
hyddf = pd.DataFrame(hyd.obsdata)
#hyddf = hyddf.loc[[True if 'cr' in i.decode() else False for i in hyddf.hydlbl]]
hyd.obsdata = hyddf.to_records(index=False).astype(hyd.obsdata.dtype)
hyd.nhyd = len(hyd.obsdata)
m.exe_name = 'mfnwt'
m.write_input()
pyemu.helpers.run('{0} {1}'.format(m.exe_name,nam_file), cwd='temp')


changing model workspace...
   temp
run():./mfnwt freyberg.nam


## Let's make some parameters. How about zones for HK and a constants for SY and RCH?

In [3]:
zn_array = np.loadtxt(os.path.join("Freyberg_truth","hk.zones"))
k_zone_dict = {k:zn_array for k in range(m.nlay)}
const_props = []
const_props.append(["upw.sy", None])
const_props.append(["rch.rech",None])
zone_props = [['upw.hk',0]]

## Maybe we ought to also treat well pumping as an uncertain parameter

In [4]:
bc_props = [["wel.flux",None]]

In [5]:
mfp = pyemu.helpers.PstFromFlopyModel(m,new_model_ws="schur_test",zone_props=zone_props,
                                          const_props=const_props,k_zone_dict=k_zone_dict,
                                          remove_existing=True,bc_props=bc_props)

2017-12-03 11:22:42.714549 starting: updating model attributes
2017-12-03 11:22:42.714671 finished: updating model attributes took: 0:00:00.000122
2017-12-03 11:22:42.714738 WARNING: removing existing 'new_model_ws

creating model workspace...
   schur_test

changing model workspace...
   schur_test
2017-12-03 11:22:42.814868 starting: writing new modflow input files
Util2d:delr: resetting 'how' to external
Util2d:delc: resetting 'how' to external
Util2d:model_top: resetting 'how' to external
Util2d:botm_layer_0: resetting 'how' to external
Util2d:botm_layer_1: resetting 'how' to external
Util2d:botm_layer_2: resetting 'how' to external
Util2d:ibound_layer_0: resetting 'how' to external
Util2d:ibound_layer_1: resetting 'how' to external
Util2d:ibound_layer_2: resetting 'how' to external
Util2d:strt_layer_0: resetting 'how' to external
Util2d:strt_layer_1: resetting 'how' to external
Util2d:strt_layer_2: resetting 'how' to external
Util2d:rech_1: resetting 'how' to external
Util2d:rech_

Util2d:rech_239: resetting 'how' to external
Util2d:rech_240: resetting 'how' to external
Util2d:rech_241: resetting 'how' to external
Util2d:rech_242: resetting 'how' to external
Util2d:rech_243: resetting 'how' to external
Util2d:rech_244: resetting 'how' to external
Util2d:rech_245: resetting 'how' to external
Util2d:rech_246: resetting 'how' to external
Util2d:rech_247: resetting 'how' to external
Util2d:rech_248: resetting 'how' to external
Util2d:rech_249: resetting 'how' to external
Util2d:rech_250: resetting 'how' to external
Util2d:rech_251: resetting 'how' to external
Util2d:rech_252: resetting 'how' to external
Util2d:rech_253: resetting 'how' to external
Util2d:rech_254: resetting 'how' to external
Util2d:rech_255: resetting 'how' to external
Util2d:rech_256: resetting 'how' to external
Util2d:rech_257: resetting 'how' to external
Util2d:rech_258: resetting 'how' to external
Util2d:rech_259: resetting 'how' to external
Util2d:rech_260: resetting 'how' to external
Util2d:rec

2017-12-03 11:22:48.073311 forward_run line:pyemu.helpers.apply_array_pars()

2017-12-03 11:22:48.073629 starting: processing obs type mflist water budget obs
run():inschek schur_test/flux.dat.ins schur_test/flux.dat
run():inschek schur_test/vol.dat.ins schur_test/vol.dat
run():inschek schur_test/flux.dat.ins schur_test/flux.dat
run():inschek schur_test/vol.dat.ins schur_test/vol.dat
2017-12-03 11:22:48.941499 forward_run line:pyemu.gw_utils.apply_mflist_budget_obs('freyberg.list',flx_filename='flux.dat',vol_filename='vol.dat',start_datetime='1/1/2015')
2017-12-03 11:22:48.941732 finished: processing obs type mflist water budget obs took: 0:00:00.868103
2017-12-03 11:22:48.941934 starting: processing obs type hyd file
Starting to read HYDMOD data from schur_test/freyberg.hyd.bin
2017-12-03 11:22:49.428885 forward_run line: pyemu.gw_utils.modflow_read_hydmod_file('freyberg.hyd.bin')
2017-12-03 11:22:49.429016 finished: processing obs type hyd file took: 0:00:00.487082
2017-12-03 11:22:4

2017-12-03 11:22:51.079872 pestcheck:cited more than once.
2017-12-03 11:22:51.080861 pestcheck:Line 10787 of file freyberg_pest.pst: model input file arr_mlt/rech0.dat_cn
2017-12-03 11:22:51.080964 pestcheck:cited more than once.
2017-12-03 11:22:51.081044 pestcheck:Line 10788 of file freyberg_pest.pst: model input file arr_mlt/rech0.dat_cn
2017-12-03 11:22:51.081245 pestcheck:cited more than once.
2017-12-03 11:22:51.081298 pestcheck:Line 10789 of file freyberg_pest.pst: model input file arr_mlt/rech0.dat_cn
2017-12-03 11:22:51.081347 pestcheck:cited more than once.
2017-12-03 11:22:51.081387 pestcheck:Line 10790 of file freyberg_pest.pst: model input file arr_mlt/rech0.dat_cn
2017-12-03 11:22:51.081429 pestcheck:cited more than once.
2017-12-03 11:22:51.081471 pestcheck:Line 10791 of file freyberg_pest.pst: model input file arr_mlt/rech0.dat_cn
2017-12-03 11:22:51.081509 pestcheck:cited more than once.
2017-12-03 11:22:51.081545 pestcheck:Line 10792 of file freyberg_pest.pst: model 

/Users/mnfienen/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/mnfienen/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


# Cool - let's take a quick look at the PST control file this made

In [6]:
inpst = mfp.pst

In [7]:
inpst.parameter_data

,parnme,partrans,parchglim,parval1,parlbnd,parubnd,pargp,scale,offset,dercom
hk0_zn1.0,hk0_zn1.0,log,factor,1.0,0.10,10.00,_znhk0,1.0,0.0,1
hk0_zn2.0,hk0_zn2.0,log,factor,1.0,0.10,10.00,_znhk0,1.0,0.0,1
hk0_zn3.0,hk0_zn3.0,log,factor,1.0,0.10,10.00,_znhk0,1.0,0.0,1
hk0_zn4.0,hk0_zn4.0,log,factor,1.0,0.10,10.00,_znhk0,1.0,0.0,1
hk0_zn5.0,hk0_zn5.0,log,factor,1.0,0.10,10.00,_znhk0,1.0,0.0,1
hk0_zn6.0,hk0_zn6.0,log,factor,1.0,0.10,10.00,_znhk0,1.0,0.0,1
sy0_cn,sy0_cn,log,factor,1.0,0.25,1.75,_cn,1.0,0.0,1
rech0_cn,rech0_cn,log,factor,1.0,0.75,1.25,_cn,1.0,0.0,1
welflux_000,welflux_000,log,factor,1.0,0.10,10.00,welflux,1.0,0.0,1


# Looks like we have multipliers on our zones. Let's set `noptmax` to -1 to calculate a Jacobian matrix and then see what `pyemu` functionality we can light up

## First, though, we can report all the control data to see `noptmax` and everything else

In [8]:
inpst.control_data.formatted_values

name
rstfle                        restart
pestmode                   estimation
npar                                9
nobs                            10598
npargp                              3
nprior                              0
nobsgp                             29
maxcompdim                          0
ntplfle                           370
ninsfle                             3
precis                         single
dpoint                          point
numcom                              1
jacfile                             0
messfile                            0
obsreref                   noobsreref
rlambda1                 2.000000E+01
rlamfac                 -3.000000E+00
phiratsuf                3.000000E-01
phiredlam                1.000000E-02
numlam                             -7
jacupdate                         999
lamforgive                 lamforgive
derforgive               noderforgive
relparmax                1.000000E+01
facparmax                1.000000E+01
facorig

In [9]:
inpst.control_data.noptmax=-1
inpst.write(os.path.join('schur_test','freyberg_pest.pst'))

## Set a flag for whether to run in parallel or not

In [12]:
parallel = True

In [13]:
if parallel==False:
    pyemu.helpers.run('pestpp freyberg_pest.pst', cwd='schur_test')

# We can also run in parallel if we want, using the `start_slaves` helper. Set the number of workers you want (all we be launched locally on your machine) and reset parallel to `True`

In [14]:
parallel = True
if parallel == True:
    num_workers = 12
    os.chdir('schur_test')
    pyemu.helpers.start_slaves('.','pestpp','freyberg_pest.pst',num_slaves=num_workers,master_dir='.')
    os.chdir('..')

master:pestpp freyberg_pest.pst /h :4004 in .
slave:pestpp freyberg_pest.pst /h localhost:4004 in ../slave_0
slave:pestpp freyberg_pest.pst /h localhost:4004 in ../slave_1
slave:pestpp freyberg_pest.pst /h localhost:4004 in ../slave_2
slave:pestpp freyberg_pest.pst /h localhost:4004 in ../slave_3
slave:pestpp freyberg_pest.pst /h localhost:4004 in ../slave_4
slave:pestpp freyberg_pest.pst /h localhost:4004 in ../slave_5
slave:pestpp freyberg_pest.pst /h localhost:4004 in ../slave_6
slave:pestpp freyberg_pest.pst /h localhost:4004 in ../slave_7
slave:pestpp freyberg_pest.pst /h localhost:4004 in ../slave_8
slave:pestpp freyberg_pest.pst /h localhost:4004 in ../slave_9
slave:pestpp freyberg_pest.pst /h localhost:4004 in ../slave_10
slave:pestpp freyberg_pest.pst /h localhost:4004 in ../slave_11


# Now, once we have a Jacobian matrix, we can create a posterior Schur complement object. If we don't pass an explicit covariance matrix, the object will use parameter bounds for the prior variance (diagonal matrix) for parameters (also note that if no `.pst` file was passed, `pyemu` will look for one with the same root name in the same location as the `.jco` file that was passed.

In [15]:
sc = pyemu.Schur(os.path.join('schur_test','freyberg_pest.jcb'))

In [16]:
sc.jco.x

array([[  1.12992135e-01,  -4.01687040e+01,  -1.39714775e+02, ...,
          0.00000000e+00,  -2.15798029e+03,   0.00000000e+00],
       [ -1.12992135e-01,  -3.97167355e+01,   6.11287451e+01, ...,
          0.00000000e+00,  -1.21924163e+04,   0.00000000e+00],
       [  0.00000000e+00,  -7.98820188e+01,  -7.85398510e+01, ...,
          0.00000000e+00,   1.74574118e+02,   0.00000000e+00],
       ..., 
       [  3.61574832e+02,   4.31720350e+04,   7.69865133e+04, ...,
          1.62708674e+04,  -8.62370438e+05,   1.07425329e+06],
       [  1.15703946e+02,   9.25631570e+02,   2.89259866e+03, ...,
         -3.87608220e+03,  -4.10749009e+03,  -3.87608220e+03],
       [  0.00000000e+00,  -1.73555919e+02,  -9.86376142e+03, ...,
         -2.19837498e+04,  -3.55789635e+03,  -3.14998209e+06]])

In [19]:
inpst.observation_data.obgnme.unique()

array(['flx_constan', 'flx_drains', 'flx_in-out', 'flx_percent',
       'flx_recharg', 'flx_storage', 'flx_total', 'flx_wells',
       'i001cr03c16', 'i001cr03c10', 'i001cr04c9', 'i001cr10c2',
       'i001cr14c11', 'i001cr16c17', 'i001cr22c11', 'i001cr23c16',
       'i001cr25c5', 'i001cr27c7', 'i001cr30c16', 'i001cr34c8',
       'i001cr35c11', 'vol_constan', 'vol_drains', 'vol_in-out',
       'vol_percent', 'vol_recharg', 'vol_storage', 'vol_total',
       'vol_wells'], dtype=object)